In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']


In [ ]:
name_dict = {'escort': "Escorting", 
             'work': "Work", 
             'school': "School", 
             'othmaint': "Other-Maintenance",
             'othdiscr': "Other-Discretionary", 
             'shopping': "Shopping",
             'atwork': "At-work Subtour", 
             'eatout': "Eat Out", 
             'social' : "Social", 
             'univ': "University"}

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# unfiltered file input

base_tour = pd.read_csv(f"{base_dir}/final_tours.csv", 
                        index_col='tour_id', 
                        usecols=['tour_id','tour_category','origin','destination','primary_purpose','number_of_participants','start','end'])

# this will be used in filtering the base tours
base_tour_idx = base_tour.index

build_tour = pd.read_csv(f"{build_dir}/final_tours.csv", 
                         index_col='tour_id', 
                         usecols=['tour_id','tour_category','origin','destination','primary_purpose','number_of_participants','start','end'])


In [ ]:
print(f"""Filtering tours by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal tours in base: {len(base_tour)}\tbuild: {len(build_tour)}""")

base_tour = base_tour[multi_filter_mazs([base_tour.origin, base_tour.destination])] # base tour in the filtered set

# special build-case filtering
build_tour = build_tour[(
    build_tour.index.isin(base_tour_idx) # originally existed in base
    & build_tour.index.isin(base_tour.index)    # and was in the current set for the base
    ) |     # OR
    ((~build_tour.index.isin(base_tour_idx)) # is a new tour 
     & multi_filter_mazs([build_tour.origin, build_tour.destination]) # and it's in this set
     )]

print(f"After filtering, tours in base: {len(base_tour)}\tbuild: {len(build_tour)}")

## Joint Tour Participation

In [ ]:
base_persons_df = pd.read_csv(
    f"{base_dir}/final_persons.csv",
    index_col="person_id",
    usecols=[
        "person_id",
        "num_joint_tours",
        'home_zone_id'
    ]
)

build_persons_df = pd.read_csv(
    f"{build_dir}/final_persons.csv",
    index_col="person_id",
    usecols=[
        "person_id",
        "num_joint_tours",
        'home_zone_id'
    ]
)

In [ ]:
print(f"Filtering persons by home MAZ. Original Persons: {len(base_persons_df)}")
base_persons_df = base_persons_df[single_filter_mazs(base_persons_df.home_zone_id)]
build_persons_df = build_persons_df[single_filter_mazs(build_persons_df.home_zone_id)]
print(f"Persons after filtering: {len(base_persons_df)}")

In [ ]:
df = pd.crosstab(
    base_persons_df.num_joint_tours,
    build_persons_df.num_joint_tours,
    rownames=['Base'],
    colnames=['Build'],
    margins=True,
    margins_name='Total'
)

df

## Joint Tour Destinations

In [ ]:
df = base_tour[base_tour.number_of_participants > 1].merge(build_tour[build_tour.number_of_participants > 1],
                                                           how='outer',
                                                           left_index=True,
                                                           right_index=True,
                                                           suffixes=('_base','_build'))

In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly


for key, value in name_dict.items():

    if len(base_tour[
        (base_tour.number_of_participants > 1)
        & (base_tour.primary_purpose == key)
        ]) == 0:
        continue
    
    print(f"#### {value}")
    df = base_tour[
        (base_tour.number_of_participants > 1)
        & (base_tour.primary_purpose == key)
        ].merge(build_tour[
                (build_tour.number_of_participants > 1)
                & (build_tour.primary_purpose == key)
            ],
                how='left',
                left_index=True,
                right_index=True,
                suffixes=('_base','_build'))
    
    df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
    df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"
    df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
    df.loc[df.destination_build.isna(),"Case"] = "Removed"

    
    df = df.Case.value_counts()
    print(str(df).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
    
    fig.show()
    display(Markdown(" "))

## Joint Tour Scheduling

In [ ]:
base_tours_df = base_tour[base_tour.number_of_participants > 1]
build_tours_df = build_tour[build_tour.number_of_participants > 1]

# translate time bin to time period index
base_tours_df['start_period'] = base_tours_df.start.apply(vh.get_time_period_index)
build_tours_df['start_period'] = build_tours_df.start.apply(vh.get_time_period_index)

base_tours_df['end_period'] = base_tours_df.end.apply(vh.get_time_period_index)
build_tours_df['end_period'] = build_tours_df.end.apply(vh.get_time_period_index)

In [ ]:
# join base and build tours on tour_id
# compare tours that are in both base and build datasets
df = pd.merge(
    base_tours_df,
    build_tours_df,
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build'),
    how='inner'
)

# get the difference in start and end times
df['start_bin_difference'] = df.start_build - df.start_base
df['end_bin_difference'] = df.end_build - df.end_base

# get the difference in start and end time periods
df['start_period_difference'] = df.start_period_build - df.start_period_base
df['end_period_difference'] = df.end_period_build - df.end_period_base

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key in ['bin', 'period']:
    for metric in ['start', 'end']:
        purpose_df = df
        display(Markdown(f"### Joint Tour {metric.capitalize()} Time {key.capitalize()} Difference"))
        purpose_df = df.groupby(f'{metric}_{key}_difference').size().reset_index()
        purpose_df.columns = [f'{metric}_{key}_difference', 'count']
        purpose_df = purpose_df.sort_values(by=f'{metric}_{key}_difference')

        fig = vh.create_bar_chart(
            source_data=purpose_df,
            source=f'{metric}_{key}_difference',
            col=f'{metric}_{key}_difference',
            plot_col='count',
            # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
        )
        fig.show()
        display(Markdown(" "))